In [2]:

import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
from vega_datasets import data as vds

In [4]:
df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
df_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [17]:
df_confirmed.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,50,50,53,59,70,75,82,114,147,177
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,502,511,581,639,639,701,773,839,825,878
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,150,150,160,178,178,200,212,226,243,266
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,99,117,129,149,149,197,238,428,566,673


In [18]:
# 1.2 Tidying the data
# Using melt() command in pandas (similar to gather() in R's tidyr)
id_list = df_confirmed.columns.to_list()[:4]
vars_list = df_confirmed.columns.to_list()[4:]
confirmed_tidy = pd.melt(df_confirmed, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='Confirmed')
deaths_tidy = pd.melt(df_deaths, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='Deaths')
recovered_tidy = pd.melt(df_recovered, id_vars=id_list,\
     value_vars=vars_list, var_name='Date', value_name='recovered')

In [23]:
from functools import reduce
data_frames = [confirmed_tidy, deaths_tidy, recovered_tidy]          
df_corona = reduce(lambda left, right: pd.merge(left, right, on =\
               id_list+['Date'], how='outer'), data_frames)

In [75]:
df_corona.tail(5)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,recovered
25755,Cayman Islands,United Kingdom,19.3133,-81.2546,3/17/20,1,1,0
25756,Reunion,France,-21.1351,55.2471,3/17/20,9,0,0
25757,NaN,Barbados,13.1939,-59.5432,3/17/20,2,0,0
25758,NaN,Montenegro,42.5000,19.3000,3/17/20,2,0,0
25759,NaN,The Gambia,13.4667,-16.6000,3/17/20,1,0,0


In [83]:
corona_sums = df_corona.groupby(['Province/State','Country/Region', 'Lat', 'Long'])['Confirmed'].sum().reset_index()


In [84]:
corona_sums.tail(50)

,Province/State,Country/Region,Lat,Long,Confirmed
267,"Spotsylvania, VA",US,38.2042,-77.6078,0
268,St Martin,France,18.0708,-63.0501,18
269,"St. Joseph, IN",US,41.6228,-86.3377,0
270,"St. Louis County, MO",US,38.6103,-90.4125,2
271,"Stanislaus, CA",US,37.5091,-120.9876,0
272,"Stark, OH",US,40.8685,-81.2519,0
273,"Suffolk County, MA",US,42.3601,-71.0589,56
274,"Suffolk County, NY",US,40.9849,-72.6151,2
275,"Sullivan, TN",US,36.4933,-82.3452,0
276,"Summit County, CO",US,39.5912,-106.0640,4


In [101]:
corona_sums = df_corona.groupby(['Province/State', 'Lat','Long'],\
                     as_index=False).agg({'Confirmed':'sum'})

In [102]:
corona_sums.head(55)

,Province/State,Lat,Long,Confirmed
0,"Adams, IN",39.8522,-77.2865,0
1,Alabama,32.3182,-86.9023,91
2,"Alachua, FL",29.7938,-82.4944,0
3,"Alameda County, CA",37.6017,-121.7195,10
4,Alaska,61.3707,-152.4044,7
5,Alberta,53.9333,-116.5765,286
6,Anhui,31.8257,117.2264,41707
7,"Anoka, MN",45.3293,-93.2197,0
8,"Arapahoe, CO",39.6203,-104.3326,0
9,Arizona,33.7298,-111.4312,96


In [81]:
import plotly_express as px
def plot_timeseries(df):
    fig = px.line(df, x='Date', y='Confirmed') 

    fig.update_layout(legend_orientation="h")
    return(fig)

fig = plot_timeseries(corona_sums)
fig.show()

In [79]:
import plotly_express as px
# Selecting only the Confirmed cases
tsmap_corona = df_corona[df_corona['Confirmed']
tsmap_corona['Date'] = tsmap_corona['Date'].astype(str)

SyntaxError: invalid syntax (<ipython-input-79-874fbbc20570>, line 4)